In [1]:
# TODO
# 1. Make map for merged dataframes
# 2. Improve data-analys:
#    2.1. Comparise speed over time summing time arrays
#    2.2. Research lengths of words and comparison eng and rus and more if will do
#    2.3. After comparise meaned data, show non-meaned to kill xor, comparise maximums of non-meaned arrays
#    2.4. Find percent when xor is more effective then division
# Amount of notes: 701 : 18; 10315 : 126; 25317 : 125; 51739 : 125.

In [2]:
from os import listdir
from utils import read_data_txt

import pandas as pd
import plotly.express as px

In [3]:
def read_csv_from_dir(path, amount_data=701):
    first = True
    
    if not path.endswith('/'):
        path += '/'
    
    df_merged = pd.DataFrame()
    
    for file_name in listdir(path):
        if file_name.find('{}'.format('_' + str(amount_data))) != -1:
            if first:
                df_merged = pd.read_csv(path + file_name)
                first = False
                continue
            
            df_merged += pd.read_csv(path + file_name)
            pass
        pass
    
    return df_merged / len(listdir(path))

In [4]:
def count_mean(hashers_dict, shift=1):
    df = pd.DataFrame()
    means = []

    hasher_names = ['Remainder_Division', 'XOR']
    modes = ['insert', 'find', 'delete']

    for mode in modes:
        for hasher_name in hasher_names:
            counter = 0
            for i in range(0, len(hashers_dict[hasher_name + '_' + mode]), shift):
                counter += 1
                if counter == 2:
                    counter = 0
                    means.append(hashers_dict[hasher_name + '_' + mode][(i - shift):i].mean())
                    pass
                pass
            df[hasher_name + '_' + mode] = pd.Series(means)
            means.clear()
            pass
        pass
    return df

In [5]:
def rename_columns(df):
    return df.rename(columns={'Remainder_Division_insert':'Деление Вставка',
                              'XOR_insert':'XOR Вставка',
                              'Remainder_Division_find':'Деление Поиск',
                              'XOR_find':'XOR Поиск',
                              'Remainder_Division_delete':'Деление Удаление',
                              'XOR_delete':'XOR Удаление'})

In [6]:
hash_tables_sizes = [701, 10315, 25317, 51739]
langs = ['eng', 'rus']
paths = ['out_data/eng', 'out_data/rus']
dict_df_merged = {}

In [7]:
# func1

for size in hash_tables_sizes:

    paths = ['out_data/eng', 'out_data/rus']
    for path in paths:
        for k in hash_tables_sizes:
            dict_df_merged.update({'{}_{}'.format(k, path.split('/')[1]): \
                                       read_csv_from_dir(path, k)})
            pass
        pass

    dict_df_merged.update({'{}_merged_langs'.format(size): \
                         (dict_df_merged.get('{}_eng'.format(size)) + \
                          dict_df_merged.get('{}_rus'.format(size))) / len(paths)})
    pass

In [19]:
dict_df_means = {}
proportions = [20, 41, 101, 207]
counter = 0
for size in hash_tables_sizes:

    dict_df_means.update({'{}_merged_means'.format(size): \
                          count_mean(dict_df_merged.get('{}_eng'.format(size)), proportions[counter])})
    counter += 1
    pass

In [20]:
size = int(input())

df_means_any = rename_columns(dict_df_means.get('{}_merged_means'.format(size)))

print(len(df_means_any))

px.line(df_means_any).update_layout(title = 'Количество данных: {}; Язык: английский'.format(size),
                 xaxis_title='Среднее каждых 35 записей',
                 yaxis_title='время, μ'.format(size)).show()

10315
126


In [ ]:
testing_data_rus = read_data_txt('test_data/test_rus.txt')
testing_data_eng = read_data_txt('test_data/test_eng.txt')

In [ ]:
count_rus = []
for word in testing_data_rus:
    count_rus.append(len(word))
    pass

count_eng = []
for word in testing_data_eng:
    count_eng.append(len(word))
    pass

In [ ]:
mean_rus = pd.Series(count_rus).mean()
mean_eng = pd.Series(count_eng).mean()

In [ ]:
print(mean_rus)
print(mean_eng)

In [ ]:
sums_rus = {}
max_rus = {}
sums_eng = {}
max_eng = {}

for size in hash_tables_sizes:
    sums_rus.update({'{}'.format(size):dict_df_merged.get('{}_rus'.format(size)).sum()})
    max_rus.update({'{}'.format(size):dict_df_merged.get('{}_rus'.format(size)).max()})
    sums_eng.update({'{}'.format(size):dict_df_merged.get('{}_eng'.format(size)).sum()})
    max_eng.update({'{}'.format(size):dict_df_merged.get('{}_eng'.format(size)).max()})

In [ ]:
px.line(sums_rus.get('51739'))

In [ ]:
px.line(max_rus.get('51739'))

In [ ]:
px.scatter(sums_eng.get('51739'))

In [ ]:
px.line(max_eng.get('51739'))

In [ ]:
# modified func1

for size in hash_tables_sizes:

    for path in paths:
        for k in hash_tables_sizes:
            dict_df_merged.update({'{}_{}'.format(k, path.split('/')[1]): \
                                       read_csv_from_dir(path, k)})
            pass
        pass

    dict_df_merged.update({'{}_merged_langs'.format(size):\
                           dict_df_merged.get('{}_{}'.format(size, langs[:1]))})
    
    for lang in langs[1:]:
        
        dict_df_merged.update({'{}_merged_langs'.format(size):\
                               dict_df_merged.get('{}_merged_langs'.format(size)) + \
                               dict_df_merged.get('{}_{}'.format(size, lang))})
    pass

for size in hash_tables_sizes:
    dict_df_merged.update({'{}_merged_langs'.format(size): \
                             dict_df_merged.get('{}_merged_langs'.format(size)) / len(paths)})
    pass